In [2]:
# Kode kraftigt inspireret fra https://www.kaggle.com/apollo2506/ship-detection-using-faster-r-cnn-part-1

# imports
%matplotlib inline
import project_path
from lib.utils import plotutils
from src.data import data_loader

def reload():
    import importlib
    importlib.reload(project_path)
    importlib.reload(plotutils)
    importlib.reload(data_loader)



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, random, pickle, json, itertools

from IPython.display import SVG
from tensorflow.keras.utils import plot_model, model_to_dot
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Add, Input, Conv2D, Dropout, Activation, BatchNormalization, MaxPooling2D, ZeroPadding2D, AveragePooling2D, Flatten, Dense)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import *

print("Ok")

Ok


In [3]:


def conv_block(X, k, filters, stage, block, s=2):
    conv_base_name = 'conv_' + str(stage) + block + '_branch'
    bn_base_name = 'bn_' + str(stage) + block + "_branch"

    F1 = filters

    X = Conv2D(filters=F1, kernel_size=(k, k), strides=(s, s),
               padding='same', name=conv_base_name + '2a')(X)
    X = BatchNormalization(name=bn_base_name + '2a')(X)
    X = Activation('relu')(X)

    return X
    pass

print("Ok")

Ok


In [38]:

def basic_model(input_shape,classes):

    X_input = Input(input_shape)

    X = ZeroPadding2D((5,5))(X_input)

    X = Conv2D(16,(3,3),strides=(2,2),name='conv1',padding="same")(X)
    X = BatchNormalization(name='bn_conv1')(X)

    # stage 2
    X = conv_block(X,3,32,2,block='A',s=1)
    X = MaxPooling2D((2,2))(X)
    X = Dropout(0.25)(X)

#     Stage 3
    X = conv_block(X,5,32,3,block='A',s=2)
    X = MaxPooling2D((2,2))(X)
    X = Dropout(0.25)(X)

#     Stage 4
    X = conv_block(X,3,64,4,block='A',s=1)
    X = MaxPooling2D((2,2))(X)
    X = Dropout(0.25)(X)

#   Output Layer
    X = Flatten()(X)
    X = Dense(64)(X)
    X = Dropout(0.5)(X)

    X = Dense(128)(X)
    X = Activation("relu")(X)

    X = Dense(classes,activation="softmax",name="fc"+str(classes))(X)

    model = Model(inputs=X_input,outputs=X,name='Feature_Extraction_and_FC')

    return model
    pass

print("Ok")

Ok


In [25]:
# Get data
shipsData = data_loader.load_ship_data_normalized()


Loading ship data from path: C:\Users\victo\Aarhus universitet\Test - Dokumenter\Elektronik\Noter Elektronik\Elektronik 6. semester\ITMAL\ITMAL - Exercises\Aflevering4\src\data\../../data/raw/shipsnet.json
Ship data loaded


In [26]:
# Train/Test/Val split
np.random.seed(42)
np.random.shuffle(shipsData.images)

np.random.seed(42)
np.random.shuffle(shipsData.labels)

total_count = len(shipsData.images)
total_count

train = int(0.7*total_count)
val = int(0.2*total_count)
test = int(0.1*total_count)

train_images, train_labels = shipsData.images[:train], shipsData.labels[:train]
val_images, val_labels = shipsData.images[train:(val+train)], shipsData.labels[train:(val+train)]
test_images, test_labels = shipsData.images[-test:], shipsData.labels[-test:]

train_images.shape, val_images.shape, test_images.shape

((2800, 80, 80, 3), (800, 80, 80, 3), (400, 80, 80, 3))

In [39]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(train_labels),
                                                 y=train_labels)
class_weights = {0: class_weights[0],
                 1: class_weights[1]}
print(class_weights)

{1: 0.6682577565632458, 0: 1.9858156028368794}


In [40]:
model = basic_model(input_shape=(80,80,3),classes=2)

#plot_model(model,to_file='basic_model.png')
#SVG(model_to_dot(model).create(prog='dot',format='svg'))

model.summary()

Model: "Feature_Extraction_and_FC"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 80, 80, 3)]       0         
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin (None, 90, 90, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 45, 45, 16)        448       
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 45, 45, 16)        64        
_________________________________________________________________
conv_2A_branch2a (Conv2D)    (None, 45, 45, 32)        4640      
_________________________________________________________________
bn_2A_branch2a (BatchNormali (None, 45, 45, 32)        128       
_________________________________________________________________
activation_32 (Activation)   (None, 45, 4

In [41]:
opt = Adam(lr=1e-3)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

checkpoint = ModelCheckpoint("model_weights.h5",monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
logs = TensorBoard("logs")

In [43]:
epochs = 10
batch_size = 16

history = model.fit(train_images,train_labels,
                   steps_per_epoch=len(train_images)//batch_size,
                   epochs=epochs,
                   verbose=1,
                   validation_data=(val_images,val_labels),
                   validation_steps=len(val_images)//batch_size,
                   callbacks=[checkpoint, logs],
                   class_weight=class_weights
                   )

Epoch 1/10
175/175 [==============================] - ETA: 0s - loss: 1.1471 - accuracy: 0.4989 

In [ ]:
class_names = ["no-ship","ship"]
plotutils.show_final_history(history)

In [ ]:
val_pred = model.predict(val_images)
val_pred = np.argmax(val_pred,axis=1)
val_pred.shape

val_actual = np.argmax(val_labels,axis=1)

cnf_mat = confusion_matrix(val_actual, val_pred)
np.set_printoptions(precision=2)

plt.figure()
plotutils.plot_confusion_matrix(cnf_mat,classes=class_names)
plt.grid(None)
plt.show();

In [ ]:
test_pred = model.predict(test_images)
test_pred = np.argmax(test_pred,axis=1)
test_pred.shape

test_actual = np.argmax(test_labels,axis=1)

cnf_mat_test = confusion_matrix(test_actual, test_pred)
np.set_printoptions(precision=2)

plt.figure()
plotutils.plot_confusion_matrix(cnf_mat_test,classes=class_names)
plt.grid(None)
plt.show()

In [ ]:
rnd_idx = random.sample(range(0,400),10)

class_labels = {i:class_name for (class_name,i) in class_name_labels.items()}
class_labels

# fig, ax = plt.subplots(2,5,figsize=(5,5))

for i,idx in enumerate(rnd_idx):

    plt.imshow(test_images[idx])
    plt.title("Actual: {}\nPredicted: {}".format(class_labels[test_actual[idx]],class_labels[test_pred[idx]]))
    plt.grid(None)
    plt.show()
    pass

In [ ]:
import inspect
dirname = os.path.dirname(os.path.abspath(inspect.stack()[0][1])) #Current folder

datapath = os.path.join(dirname, '../../models/ship-model.h5')
model.save(datapath)